<a href="https://colab.research.google.com/github/Yewon9/STT/blob/main/Wav2Vec2_Fine_Tuning_Disordered_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets soundfile pydub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import io
import soundfile as sf
from pydub import AudioSegment
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import torch

# DATA

In [27]:
df = pd.read_csv('disorder_eda.csv')
df

,Path,Text,Audio_Length,Cleaned_Text,Text_Length
0,/content/drive/MyDrive/project/data/Training/w...,미영이랑 나연이는 단짝입니다. 미영이와 나연이는 노래하며 놉니다. 마루 위에 나란히...,772.163,미영이랑 나연이는 단짝입니다. 미영이와 나연이는 노래하며 놉니다. 마루 위에 나란히...,2038
1,/content/drive/MyDrive/project/data/Training/w...,거울 안경 전화 신발 나무 가위 사과 모자 풍선 당근 연필 기차 날개 창문 동생 약...,206.577,거울 안경 전화 신발 나무 가위 사과 모자 풍선 당근 연필 기차 날개 창문 동생 약...,215
2,/content/drive/MyDrive/project/data/Training/w...,나는 바지를 입고 단추를 채웁니다. 책상 위에 가방이 있습니다. 가방에 사탕과 연필...,709.480,나는 바지를 입고 단추를 채웁니다. 책상 위에 가방이 있습니다. 가방에 사탕과 연필...,335
3,/content/drive/MyDrive/project/data/Training/w...,새 해 배 개 집 일 입 이 꿀 불 귤 물 피 쥐 비 귀 콩 돈 공 손 풀 붓 뿔 ...,1490.877,새 해 배 개 집 일 입 이 꿀 불 귤 물 피 쥐 비 귀 콩 돈 공 손 풀 붓 뿔 ...,299
4,/content/drive/MyDrive/project/data/Training/w...,바지 단추 책상 가방 사탕 연필 자동차 동물원 엄마 뽀뽀 호랑이 꼬리 코끼리 땅콩 ...,276.963,바지 단추 책상 가방 사탕 연필 자동차 동물원 엄마 뽀뽀 호랑이 꼬리 코끼리 땅콩 ...,89
...,...,...,...,...,...
309,/content/drive/MyDrive/project/data/Training/w...,제 나이는 서른 일곱 살입니다. 저는 용 띠입니다. 제가 태어난 곳은 동해시이고 주...,920.580,제 나이는 서른 일곱 살입니다. 저는 용 띠입니다. 제가 태어난 곳은 동해시이고 주...,724
310,/content/drive/MyDrive/project/data/Training/w...,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,3816.600,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4211
311,/content/drive/MyDrive/project/data/Training/w...,거짓말을 하는 것이 분명합니다. 고기는 좋아하지만 생선은 싫어한다. 기차를 탈 분은...,2753.132,거짓말을 하는 것이 분명합니다. 고기는 좋아하지만 생선은 싫어한다. 기차를 탈 분은...,200
312,/content/drive/MyDrive/project/data/Training/w...,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4320.615,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4190


In [28]:
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['Path', 'Text', 'Audio_Length', 'Cleaned_Text', 'Text_Length'],
    num_rows: 314
})


# Wav2Vec2

In [29]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeque

# 음성 파일 30초 단위 분할 및 텍스트 처리

In [30]:
def filter_short_audio(batch):
    audio = AudioSegment.from_wav(batch["Path"])
    if len(audio) < 1000:  # 1초 미만의 음성 파일 필터링
        return False
    return True

In [31]:
def sample_audio_first_1_min(path, sample_length=60):
    audio = AudioSegment.from_wav(path)
    sample_audio = audio[:sample_length * 1000]  # 첫 60초를 샘플링
    return sample_audio

In [32]:
def speech_file_to_array_fn(batch):
    # 각 파일에서 1분(60초)만 샘플링
    sampled_audio = sample_audio_first_1_min(batch["Path"], sample_length=60)

    # STT 수행
    with io.BytesIO() as buffer:
        sampled_audio.export(buffer, format="wav")
        buffer.seek(0)
        speech_array, sampling_rate = sf.read(buffer)
        input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_values

        # STT 모델 예측
        with torch.no_grad():
            logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # 텍스트 변환
        transcription = processor.batch_decode(predicted_ids)[0]

    # 결과 저장
    batch["Cleaned_Text"] = transcription
    return batch

In [ ]:
dataset = dataset.filter(filter_short_audio)

Filter:   0%|          | 0/314 [00:00<?, ? examples/s]

In [25]:
dataset = dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/314 [00:00<?, ? examples/s]

RuntimeError: Calculated padded input size per channel: (2). Kernel size: (10). Kernel size can't be greater than actual input size

# Fine Tuning을 위한 학습 준비

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
data_collator = DataCollatorWithPadding(processor)

In [ ]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-finetune-disorder",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    learning_rate=1e-4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    report_to="none"  # WandB 비활성화
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)

In [ ]:
trainer.train()